In [ ]:
import codecs
import json
import joblib
import re
from functools import reduce
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import codecs
import json
import joblib
import re
from functools import reduce
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import trange, tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
import pandas as pd
from sklearn.metrics import accuracy_score
import torch.nn as nn

In [ ]:
import random
import time
import gc

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.9MB 8.5MB/s 
     |████████████████████████████████| 3.2MB 56.3MB/s 
     |████████████████████████████████| 890kB 62.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c785bd2380b3aee65374bd485a41ee67b327bcbe8e9a03db5503ec8654dd2220
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import pipeline

In [ ]:
!pip install nlp

     |████████████████████████████████| 1.7MB 7.7MB/s 
     |████████████████████████████████| 245kB 49.4MB/s 


In [ ]:
def standardize(txt):
    return txt.replace('\n', '').replace('@highlight', '')

def standardize_new(txt):
    txt = txt.replace('\'\'', '\'').replace('@header', '').replace('\n', ' ').replace('@highlight', '.').replace('. .', '.').replace(' .', '.').replace('.', '. ').replace('  ', ' ').replace('  ', ' ').strip()
    if txt[-1] != '.':
        txt += '.'
    return txt

In [ ]:
def to_t5_format(item, translate=True):
    entities = list(set([item["passage"]["text"][e["start"]:e["end"]] for e in item["passage"]["entities"]]))
    en_entities = [answers_en[item["passage"]["text"]][e] if translate and e in answers_en[item["passage"]["text"]] else e for e in entities]
    translated_text = texts_en[item["passage"]["text"]] if translate else item["passage"]["text"]
    tanslated_query = questions_en[item["passage"]["text"]] if translate else item["passage"]["text"]
    tanslated_query = "." if tanslated_query == "" else tanslated_query
    translated_text = "." if translated_text == "" else tanslated_query
    final_text = "record query: " + (tanslated_query + ". entities: " if tanslated_query[-1] != "." else tanslated_query + " entities: ") + ", ".join(en_entities) + " passage: " + (translated_text + "." if translated_text[-1] != "." else translated_text)
    return final_text

In [ ]:
def check_errors(arr):
    ixs = []
    for i, item in enumerate(arr):
        try:
          questions_en[standardize(item["qas"][0]["query"])]
        except:
          ixs.append(i)
    return ixs

In [ ]:
def save_solution(arr, name=None):
    name = (str(int(time.time())) if name == None else name) + ".jsonl"
    with open(name, 'w') as fp:
          for i, o in enumerate(arr):
              slovar = {"idx":i, "text":o}
              fp.write(json.dumps(slovar, ensure_ascii=False)+"\n")
    print("solution was saved to ", name)

In [ ]:
import codecs
import json
def read_jsonl(path):
    with codecs.open(path, encoding='utf-8-sig') as reader:
            lines = reader.read().split("\n")
            lines = list(map(json.loads, filter(None, lines)))
    return lines

In [ ]:
train = read_jsonl("/content/drive/MyDrive/final/rucos_train.jsonl")

In [ ]:
test = read_jsonl("/content/drive/MyDrive/final/rucos_test.jsonl")

In [ ]:
val = read_jsonl("/content/drive/MyDrive/final/rucos_val.jsonl")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)\

Mounted at /content/drive


In [ ]:
answers_en = json.loads(open("/content/drive/MyDrive/final/answers_n.json").read())
questions_en = json.loads(open("/content/drive/MyDrive/final/questions_n.json").read())
texts_en = json.loads(open("/content/drive/MyDrive/final/texts_n.json").read())

In [ ]:
questions_en

In [ ]:
answers_en.keys()

In [ ]:
texts_en[ standardize_new(test[537]["passage"]["text"])]

In [ ]:
test_t5_data = list(map(to_t5_format, test))

In [ ]:
train_t5_data = list(map(to_t5_format, train))

In [ ]:
val_t5_data = list(map(to_t5_format, val))

In [ ]:
test_t5_data

In [ ]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5TokenizerFast.from_pretrained('t5-base')

In [ ]:
time.sleep(100)

In [ ]:
record_train = json.loads(open("/content/drive/MyDrive/final/train.json").read())

In [ ]:
pipe = pipeline("fill-mask")

In [ ]:
now = 1

In [ ]:
pipe(to_t5_format(record_train["data"][now]).replace("@placeholder", "[MASK]"), targets = list(set([record_train["data"][now]["passage"]["text"][e["start"]:e["end"]] for e in record_train["data"][now]["passage"]["entities"]])))

In [ ]:
record_train["data"][now]["qas"][now]["answers"]

[{'end': 92, 'start': 80, 'text': 'Simon Bolivar'},
 {'end': 466, 'start': 460, 'text': 'Bolivar'},
 {'end': 628, 'start': 622, 'text': 'Bolivar'},
 {'end': 762, 'start': 750, 'text': 'Simon Bolivar'},
 {'end': 851, 'start': 845, 'text': 'Bolivar'},
 {'end': 968, 'start': 962, 'text': 'Bolivar'}]

In [ ]:
pipe()

In [ ]:
model.to("cuda")

In [ ]:
!pip install fuzzywuzzy

In [ ]:
torch.cuda.empty_cache()
gc.collect()

168

In [ ]:
from fuzzywuzzy import fuzz

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def give_entity(position, text):
  return text[position["start"]:position["end"]]

In [ ]:
def fix_placeholder(string):
  return string.replace('@Placeholder', '@placeholder').replace('@ placeholder', '@placeholder').replace('@ Placeholder', '@placeholder')

In [ ]:
simillar_to = {}

def find_simillar(answer):
    global simillar_to
    if answer in simillar_to:
        print(simillar_to[answer], "simillar to", answer)
        return
    maxx = 0
    most_simillar = ""
    for key in answers_ru_en:
        ratio = fuzz.ratio(key, answer)
        if ratio > 95:
            most_simillar = key
            maxx = ratio
            break
        elif ratio > maxx:
            most_simillar = key
            maxx = ratio

    simillar_to[answer] = most_simillar
    print(most_simillar, "simillar to", answer)

In [ ]:
def find_closest_string(string, arr):
    maxx = 0
    most_simillar = ""
    for w in arr:
        if w == string:
            return string
        ratio = fuzz.ratio(string, w)
        if ratio > maxx:
            most_simillar = w
            maxx = ratio
    return most_simillar

In [ ]:
def choose_russian(en_string, rus_arr):
    for r in rus_arr:
        if answers_en[r] == en_string:
            return r

In [ ]:
test_t5_datal = [t.lower() for t in test_t5_data]

In [ ]:
test_tokens = tokenizer(test_t5_data, padding=True, truncation=True, return_tensors="pt").input_ids

In [ ]:
test_preds = []
batch_size = 32
for i in trange(0, len(test_tokens), batch_size):
    outputs = model.generate(torch.tensor(test_tokens[i:i+batch_size]).to("cuda"))
    tokenslist = list(map(list,outputs.cpu().numpy()))
    test_preds.extend([tokenizer.decode(tl).replace("<pad>", "").replace("</s>", "").replace(" ", "", 1) for tl in tokenslist])
    if i % 64 == 0:
        torch.cuda.empty_cache()
        gc.collect()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
with open("preds.txt", "w") as f:
    f.write("\n".join(test_preds))

In [ ]:
got = [find_closest_string(s, [answers_en[test[i]["passage"]["text"]][e] if e in answers_en[test[i]["passage"]["text"]] else e for e in list(set([test[i]["passage"]["text"][e["start"]:e["end"]] for e in test[i]["passage"]["entities"]]))] ) for i,s in enumerate(test_preds)]

In [ ]:
reverse_dict = lambda x: {k:i for i,k in x.items()}

In [ ]:
true_answers = [reverse_dict(answers_en[test[i]["passage"]["text"]])[a] if a in reverse_dict(answers_en[test[i]["passage"]["text"]]) else a for i, a in enumerate(got)]

In [ ]:
answers_ru_en = {k:i for i,k in answers_en.items()}

In [ ]:
for p in test_preds:
    if p not in answers_ru_en:
        find_simillar(p)

In [ ]:
test_preds_ru = [answers_ru_en[got[i]] for i in range(len(got))]

In [ ]:
test_preds_ru = [answers_ru_en[test_preds[i]] if test_preds[i] in answers_ru_en else answers_ru_en[simillar_to[test_preds[i]]] for i in range(len(test_preds))]

In [ ]:
answers_en["Ариану Гранде"]

'Ariana Grande'

In [ ]:
from collections import Counter

In [ ]:
Counter([x in list(set([test[i]["passage"]["text"][e["start"]:e["end"]] for e in test[i]["passage"]["entities"]] )) for i, x in enumerate(test_preds_ru)])

NameError: ignored

In [ ]:
test_preds_ru = [x if x != "NOTFOUND" else give_entity(random.choice(test[i]["passage"]["entities"]), test[i]["passage"]["text"]) for i, x in enumerate(test_preds_ru)]

In [ ]:
save_solution(true_answers)

solution was saved to  1614674337.jsonl
